In [44]:
import pandas as pd
import numpy as np
from parsivar import FindStems, Tokenizer, Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [45]:
df = pd.read_csv("Snappfood - Sentiment Analysis.csv", encoding="UTF-8", sep='\t')
df.head()

,Unnamed: 0,comment,label,label_id
0,NaN,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,NaN,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,NaN,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,NaN,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,NaN,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0


In [46]:
df.columns

Index(['Unnamed: 0', 'comment', 'label', 'label_id'], dtype='object')

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  520 non-null    object 
 1   comment     70000 non-null  object 
 2   label       70000 non-null  object 
 3   label_id    69480 non-null  float64
dtypes: float64(1), object(3)
memory usage: 2.1+ MB


In [48]:
df.isnull().sum()

Unnamed: 0    69480
comment           0
label             0
label_id        520
dtype: int64

In [49]:
mask = df['label_id'].isnull()
df.loc[mask, 'label_id'] = df.loc[mask].apply(lambda row: 0 if row['label'] == 'HAPPY' else 1, axis=1)

In [50]:
X = df['comment']
y = df["label_id"]

In [51]:
from string import punctuation

punc = punctuation + "۱۲۳۴٤۵۶٦۷۸۹۰1234567890،؛"
stop_words = open('stopwords-fa.txt', 'r', encoding='utf-8').read().splitlines()

normalizer = Normalizer()
tokenizer = Tokenizer()
stemmer = FindStems()

def text_preprocess(X):
    normalized = normalizer.normalize(X)
    tokens = tokenizer.tokenize_words(normalized)
    words = [stemmer.convert_to_stem(w) for w in tokens if w not in stop_words and w not in punc and not w.isdigit()]
    tokens = " ".join(words)
    return tokens

X = X.apply(text_preprocess)

In [52]:
X.head()

0            حیف وقت نوشت&نویس سرویس دهیتون‌شده‌افتضاح
1    قرار‌بود ساعته برسه نیم ساعت زود موقع دید&بین ...
2    قیمت مدل کیفیت سازگاری نداره ظاهر فریبنده داره...
3    عالللی‌بود درست اندازه کیفیت امیداورم کیفیتتون...
4                                   شیرینی نیل مدل‌بود
Name: comment, dtype: object

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [62]:
model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.87      0.76      0.81      6962
         1.0       0.79      0.88      0.83      7038

    accuracy                           0.82     14000
   macro avg       0.83      0.82      0.82     14000
weighted avg       0.83      0.82      0.82     14000

